# 18/10/2024

Tenemos varios campos:

COUNTRY:país 
AMOUNT: un volumen de ventas en litros del producto.
SUBBRAND: el producto de la venta
YEAR: Año 
MONTH: Mes
SCENARIO: (dos valores)
- AI forecast: venta predicha en algún momento
- Actual: que se ha producido de verdad
Forecast: Si scenario es actual, este valor es Null. Indica el mes en periodos. Ejemplos:
- AI_P02F es Enero (a mes vencido). La prediccion se hizo a finales de enero
- AI_PF: Es como si fuera el mes de diciembre (AI_P13F) pero este dato no existe.

## Bibliotecas y Funciones

In [80]:
import pandas as pd
import plotly.express as px

In [81]:
meses_dict = {
    'AI_P01F': 1,
    'AI_P02F': 2,
    'AI_P03F': 3,
    'AI_P04F': 4,
    'AI_P05F': 5,
    'AI_P06F': 6,
    'AI_P07F': 7,
    'AI_P08F': 8,
    'AI_P09F': 9,
    'AI_P10F': 10,
    'AI_P11F': 11,
    'AI_P12F': 12,
    'AI_PF': 12
}

## Carga de datos

In [82]:
# Cargar el archivo CSV
ruta_archivo = 'datos_ejercicio_ventas.csv'  # Reemplaza 'nombre_del_archivo.csv' por el nombre de tu archivo o la ruta completa
df = pd.read_csv(ruta_archivo)

# Mostrar las primeras filas del dataframe
print(df.head())


         COUNTRY        SUBBRAND  YEAR  MONTH     SCENARIO FORECAST  \
0       Portugal     Lipton (L3)  2023     12  AI_forecast  AI_P02F   
1  Great Britain     Lipton (L3)  2023     12  AI_forecast  AI_P10F   
2          Spain  Pepsi Max (L3)  2023     12  AI_forecast  AI_P09F   
3  Great Britain        7up (L3)  2024     12  AI_forecast  AI_P10F   
4        Hungary     Lipton (L3)  2023      9  AI_forecast  AI_P03F   

   FORECAST_YEAR         AMOUNT  
0         2023.0  754356.237194  
1         2023.0  560030.558029  
2         2023.0   88501.980847  
3         2023.0  363224.511516  
4         2023.0  396176.120491  


## Exploración

Responder a:
1. nº de actuals y forecast
2. horizonte de previsión de los forecast: Nº de puntos que predices
3. nº de paises y de productos:
4. Que histórico de actuals o de forecast
5. Forecast distintos

In [83]:
print(df.dtypes)

COUNTRY           object
SUBBRAND          object
YEAR               int64
MONTH              int64
SCENARIO          object
FORECAST          object
FORECAST_YEAR    float64
AMOUNT           float64
dtype: object


### 1 Nº de actuals, Nº de forecast

In [84]:
conteo_scenarios = df['SCENARIO'].value_counts()
# Crear etiquetas personalizadas para mostrar el nombre y el número total
labels = [f'{scenario}: {count}' for scenario, count in zip(conteo_scenarios.index, conteo_scenarios.values)]

# Crear el gráfico de tarta con plotly.express
fig = px.pie(values=conteo_scenarios.values, 
             names=labels, 
             title="Distribución de Actuals y AI forecast",
             hole=0.3)  # 'hole=0.3' crea un gráfico tipo 'donut'

# Mostrar el gráfico
fig.show()

### 2 Horizonte de predicción

In [85]:
# Aplicar la transformación a la columna 'FORECAST', usando el diccionario
df1 = df
df1['FORECAST'] = df['FORECAST'].map(meses_dict)

# Convertir YEAR, MONTH, FORECAST, y FORECAST_YEAR a enteros (si no lo son ya)
df1['YEAR'] = pd.to_numeric(df1['YEAR'], errors='coerce').astype('Int64')
df1['MONTH'] = pd.to_numeric(df1['MONTH'], errors='coerce').astype('Int64')
df1['FORECAST'] = pd.to_numeric(df1['FORECAST'], errors='coerce').astype('Int64')
df1['FORECAST_YEAR'] = pd.to_numeric(df1['FORECAST_YEAR'], errors='coerce').astype('Int64')

# Crear nueva columna de fecha con YEAR y MONTH, asegurando que sean números válidos
df1['Fecha'] = pd.to_datetime(df1['YEAR'].astype(str) + '-' + df1['MONTH'].astype(str) + '-28', errors='coerce')

# Crear nueva columna de fecha con FORECAST_YEAR y FORECAST, asegurando que sean números válidos
df1['Fecha_forecast'] = pd.to_datetime(df1['FORECAST_YEAR'].astype(str) + '-' + df1['FORECAST'].astype(str) + '-28', errors='coerce')

# Eliminar las columnas YEAR, MONTH, FORECAST y FORECAST_YEAR
df1 = df1.drop(columns=['YEAR', 'MONTH', 'FORECAST', 'FORECAST_YEAR'])

# Mostrar las primeras filas del dataframe para verificar
print(df1.head())


         COUNTRY        SUBBRAND     SCENARIO         AMOUNT      Fecha  \
0       Portugal     Lipton (L3)  AI_forecast  754356.237194 2023-12-28   
1  Great Britain     Lipton (L3)  AI_forecast  560030.558029 2023-12-28   
2          Spain  Pepsi Max (L3)  AI_forecast   88501.980847 2023-12-28   
3  Great Britain        7up (L3)  AI_forecast  363224.511516 2024-12-28   
4        Hungary     Lipton (L3)  AI_forecast  396176.120491 2023-09-28   

  Fecha_forecast  
0     2023-02-28  
1     2023-10-28  
2     2023-09-28  
3     2023-10-28  
4     2023-03-28  


In [86]:
# 